# Refinitiv Data Library for Python
## Content - Streaming Price Cache - Synchronous calls

This notebook demonstrates how to use the Streaming Price interface to retrieve streaming data from the Refinitiv Data Platform.

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session('rdp')

<refinitiv.data.session._platform_session.Definition object at 0x21b2ff2fc10 {session_name='session.platform.1'}>

## Define and Open a cache for Streaming data
Define a Pricing object from the Streaming data feed - note the **get_stream()** call

In [3]:
# Define our Streaming Price object
streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY='],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
streaming.open()
# When you want the current price, get a snapshot
streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1845,1.1849
1,GBP=,1.3922,1.3923
2,JPY=,109.45,109.46


#### Call get_snapshot() again, prices may change (assuming instruments are being traded/quoted)

In [4]:
snap = streaming.get_snapshot()
display(snap)

,Instrument,BID,ASK
0,EUR=,1.1844,1.1848
1,GBP=,1.392,1.3924
2,JPY=,109.45,109.46



### Alternative ways of accessing instruments + values

#### Direct Access to fields
We can directly access the cache without the need to call **get_snapshot()**

In [5]:
# Directly access the latest 'BID' price for the EURO
streaming['EUR=']['BID']

1.1844

In [6]:
## Access the individual StreamingPrice object for GBP
gbp = streaming['GBP=']
# and then use object to access individual fields
gbp['ASK']

1.3923

In [7]:
# Will be different from above if price has changed
gbp['ASK']

1.3921

#### Iterate on fields

In [8]:
print('JPY=')
for field_name, field_value in streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 109.44
	ASK : 109.47


#### Iterate on Streaming instruments and fields

In [9]:
for instrument in streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.1844
	ASK : 1.1848
GBP=
	BID : 1.3918
	ASK : 1.3922
JPY=
	BID : 109.45
	ASK : 109.46


### Close the Streaming Items when no longer required

In [10]:
streaming.close()

<StreamState.Closed: 1>

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [11]:
# Define our Streaming Price object
mixed = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
state = mixed.open()
mixed.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1845,1.1846
1,GBP=,1.3919,1.3922
2,BADRIC,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [12]:
mixed['BADRIC'].status

{'status': <StreamState.Closed: 1>,
 'code': 'NotFound',
 'message': '**The record could not be found'}

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

## Close the session

In [13]:
close_session()